<a href="https://colab.research.google.com/github/abhijithnadig/abhijithnadig/blob/main/Spatial_Computing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import networkx as nx

################################################################################
# 1) Classes Area & Region
################################################################################
class Area:
    def __init__(self, area_id, geometry=None):
        self.area_id = area_id
        self.geometry = geometry
        self.nested_ids = set()  # if this area encloses smaller areas

    def __repr__(self):
        return f"Area({self.area_id})"

class Region:
    def __init__(self, target_cardinality):
        self.target_c = target_cardinality
        self.area_ids = set()   # IDs of areas assigned to this region

    def __repr__(self):
        return f"Region(size={len(self.area_ids)}/{self.target_c})"

    def is_valid(self):
        return len(self.area_ids) == self.target_c

################################################################################
# 2) Region Growing Phase
#
#    Grows a region up to region.target_c by picking a random seed from the
#    unassigned set and adding random boundary neighbors.
################################################################################
def grow_region(region, A_unassigned, G, seed_area=None, max_region_retries=10):
    """
    Attempt to build 'region' to match region.target_c by
    randomly picking a seed and adding boundary neighbors.
    """
    for attempt in range(max_region_retries):
        region.area_ids.clear()  # fresh
        # If no explicit seed_area or we are retrying, pick a random from A_unassigned
        chosen_seed = seed_area
        if chosen_seed is None or attempt > 0:
            if not A_unassigned:
                return False, region, A_unassigned
            chosen_seed = random.choice(list(A_unassigned))

        region.area_ids.add(chosen_seed)
        A_unassigned.remove(chosen_seed)

        # boundary is the set of unassigned neighbors of the region
        boundary = set()
        for nb in G.neighbors(chosen_seed):
            if nb in A_unassigned:
                boundary.add(nb)

        # Expand region until it reaches target_c or runs out of boundary
        while len(region.area_ids) < region.target_c and boundary:
            pick = random.choice(list(boundary))
            region.area_ids.add(pick)
            A_unassigned.remove(pick)
            boundary.remove(pick)
            # add new neighbors to boundary
            for nb2 in G.neighbors(pick):
                if nb2 in A_unassigned:
                    boundary.add(nb2)

        # check if success
        if len(region.area_ids) == region.target_c:
            return True, region, A_unassigned
        else:
            # revert: put all region areas back to unassigned
            A_unassigned |= region.area_ids

    # If we exit the loop, we failed
    region.area_ids.clear()
    return False, region, A_unassigned

################################################################################
# 3) Region Merging Phase
#
#    Ensures that the leftover unassigned areas A_unassigned remain in
#    one connected component by merging smaller leftover components
#    into the newly grown region. This might make the region too big,
#    so we fix that in the splitting phase.
################################################################################
def region_merging_phase(region, A_unassigned, G):
    if not A_unassigned:
        return region, A_unassigned
    # find connected components among A_unassigned
    subG = G.subgraph(A_unassigned)
    components = list(nx.connected_components(subG))
    # if only 1 leftover component => fine
    if len(components) <= 1:
        return region, A_unassigned

    # keep largest, merge all other components into the region
    largest_comp = max(components, key=len)
    for comp in components:
        if comp is largest_comp:
            continue
        # merge comp into region
        region.area_ids |= comp
        # remove comp from unassigned
        A_unassigned -= comp

    return region, A_unassigned

################################################################################
# 4) Region Splitting Phase
#
#    If the region is now too big, we remove random boundary areas from it
#    (and also remove their nested areas if relevant). If it becomes
#    disconnected, we only keep the largest connected sub-part.
#    If the region is too small, we do an "expansion" by pulling
#    boundary areas from A_unassigned into the region.
################################################################################
def region_splitting_phase(region, A_unassigned, G, parent_degree_map, max_remove=None):
    target_c = region.target_c
    cur_size = len(region.area_ids)
    # trivial case
    if cur_size == target_c:
        return True

    #------ 1) shrinking step if region > target
    if cur_size > target_c:
        excess = cur_size - target_c
        tries = 0
        while excess>0 and region.area_ids:
            tries += 1
            if max_remove and tries>max_remove:
                return False  # give up
            boundary_r = compute_region_boundary(region, A_unassigned, G)
            if not boundary_r:
                return False
            area_id = random.choice(list(boundary_r))
            # if it's a parent area => remove it + nested
            # skipping nested logic for brevity, or do:
            #   if parent_degree_map.get(area_id,0)>0: ...

            # remove from region
            region.area_ids.remove(area_id)
            A_unassigned.add(area_id)
            excess -= 1

            # check connectivity
            comps = list(nx.connected_components(G.subgraph(region.area_ids)))
            if len(comps) > 1:
                bigcomp = max(comps, key=len)
                for ccomp in comps:
                    if ccomp is bigcomp:
                        continue
                    region.area_ids -= ccomp
                    A_unassigned |= ccomp
                    excess -= len(ccomp)
            if not region.area_ids:
                return False  # region is empty => fail

        # re-check final size after shrinking
        if len(region.area_ids) == target_c:
            return True
        if len(region.area_ids) < target_c:
            # proceed to expansion step below
            pass

    #------ 2) expansion step if region < target
    if len(region.area_ids) < target_c:
        needed = target_c - len(region.area_ids)
        while needed>0 and A_unassigned:
            boundary_u = compute_unassigned_boundary(A_unassigned, G)
            # remove articulation points from boundary
            articulation_ids = set(nx.articulation_points(G.subgraph(A_unassigned)))
            boundary_no_artic = [b for b in boundary_u if b not in articulation_ids]
            if not boundary_no_artic:
                return False

            pick = random.choice(boundary_no_artic)
            region.area_ids.add(pick)
            A_unassigned.remove(pick)
            needed -= 1

        # final check
        return (len(region.area_ids) == target_c)

    return (len(region.area_ids)==target_c)

################################################################################
# Helpers for region_splitting_phase
################################################################################
def compute_region_boundary(region, A_unassigned, G):
    """
    A region boundary area is any area in region that has a neighbor in A_unassigned.
    """
    boundary = set()
    for a_id in region.area_ids:
        for nb in G[a_id]:
            if nb in A_unassigned:
                boundary.add(a_id)
                break
    return boundary

def compute_unassigned_boundary(A_unassigned, G):
    """
    Unassigned boundary: any area in A_unassigned with a neighbor not in A_unassigned.
    """
    boundary = set()
    for a_id in A_unassigned:
        for nb in G[a_id]:
            # if neighbor is not in A_unassigned => this a_id is boundary
            if nb not in A_unassigned:
                boundary.add(a_id)
                break
    return boundary

################################################################################
# 5) Building a single feasible solution with PRRP
################################################################################
def build_single_solution(area_ids, p_cardinalities, G, parent_degree_map,
                          max_solution_attempts=10):
    """
    Build exactly 1 feasible sample solution with the PRRP approach.
    area_ids: list of all area IDs
    p_cardinalities: e.g. [3,3,2], sum = len(area_ids)
    G: adjacency graph
    parent_degree_map: {area_id -> # of nested areas}
    max_solution_attempts: how many times we attempt building a full solution
    """
    n = len(area_ids)
    for attempt in range(max_solution_attempts):
        A_unassigned = set(area_ids)
        all_regions = []
        success = True

        # we build p regions in descending cardinality order
        # typically you'd do that: p_card_sorted = sorted(..., reverse=True)
        # but let's assume p_cardinalities is already sorted (or do it here).
        p_card_sorted = sorted(p_cardinalities, reverse=True)

        for c_val in p_card_sorted:
            new_region = Region(c_val)
            # Grow region
            ok_grow, new_region, A_unassigned = grow_region(new_region, A_unassigned, G)
            if not ok_grow:
                success = False
                break

            # Merge leftover comps
            new_region, A_unassigned = region_merging_phase(new_region, A_unassigned, G)

            # Split if overshoot
            ok_split = region_splitting_phase(new_region, A_unassigned, G, parent_degree_map)
            if not ok_split or not new_region.is_valid():
                success = False
                break

            all_regions.append(new_region)

        if success and len(all_regions) == len(p_card_sorted):
            return True, all_regions

    # if we fail all attempts
    return False, []

################################################################################
# 6) The prrp_solver function to build M solutions (in a loop)
################################################################################
def prrp_solver(area_list, p_cardinalities, M, q_cores=1):
    """
    A simplified version that does everything in a loop.
    Usually you'd parallelize with q_cores, but we'll do sequential for clarity.

    area_list: list[Area]
    p_cardinalities: e.g. [3,3,2]
    M: how many solutions to build
    q_cores: # of CPU cores (unused in this simple example)

    Returns: A list of solutions (each solution is a list of Region objects).
             If building a particular solution fails, we put None in that slot.
    """
    # Build adjacency from scratch if needed, or assume we have G & parent_degree_map
    # For simplicity, let's see if we can detect adjacency from area_list or skip
    # We won't do it automatically here. We'll rely on a separate function.
    # But let's just see if there's a 'G' attribute. We'll skip that and do a simpler approach:

    # We'll do a cheat: the user must pass in 'area_list' where
    # area_list[0].some_graph / or we can't proceed. Let's just say we do it outside.
    # This function is typically matched with 'build_single_solution(...)' above
    # that requires G, parent_degree_map. We'll do a hack: store them as globals:
    raise NotImplementedError("In this example, please call build_single_solution directly.")
    # or adapt to your environment.

################################################################################
# 7) A self-contained function that shows how to do everything:
#    build a TINY adjacency and run build_single_solution for M solutions
################################################################################
def main_test():
    # 1) Create 8 areas
    area_list = []
    for i in range(1, 9):
        area_list.append(Area(i))

    # 2) Build a tiny adjacency dictionary
    adjacency_dict = {
        1: [2],
        2: [1,3,4],
        3: [2,4],
        4: [2,3,5],
        5: [4,6],
        6: [5,7,8],
        7: [6],
        8: [6]
    }

    # Build an nx Graph
    G = nx.Graph()
    for a in area_list:
        G.add_node(a.area_id)
    for node_id, neighbors in adjacency_dict.items():
        for nb in neighbors:
            G.add_edge(node_id, nb)

    # No nested polygons => parent_degree_map=0
    parent_degree_map = {a.area_id: 0 for a in area_list}

    # 3) We'll try p=3 with cardinalities [3,3,2] => total 8
    p_cardinalities = [3,3,2]
    M = 2  # we want 2 sample solutions

    solutions = []
    for _ in range(M):
        ok, regs = build_single_solution(
            area_ids=[a.area_id for a in area_list],
            p_cardinalities=p_cardinalities,
            G=G,
            parent_degree_map=parent_degree_map,
            max_solution_attempts=10
        )
        if ok:
            solutions.append(regs)
        else:
            solutions.append(None)

    # 4) Print results
    for i, sol in enumerate(solutions):
        if sol is None:
            print(f"\nSolution #{i}: FAILED.")
        else:
            print(f"\nSolution #{i}: SUCCESS => # of regions = {len(sol)}")
            for r_idx, region in enumerate(sol):
                # region.target_c, region.area_ids
                print(f"  Region {r_idx} [target={region.target_c}] => {sorted(region.area_ids)}")

################################################################################
# If you run this file directly, we'll call main_test().
################################################################################
if __name__ == "__main__":
    main_test()



Solution #0: SUCCESS => # of regions = 3
  Region 0 [target=3] => [6, 7, 8]
  Region 1 [target=3] => [3, 4, 5]
  Region 2 [target=2] => [1, 2]

Solution #1: SUCCESS => # of regions = 3
  Region 0 [target=3] => [6, 7, 8]
  Region 1 [target=3] => [3, 4, 5]
  Region 2 [target=2] => [1, 2]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import random
import networkx as nx
import geopandas as gpd
from shapely.geometry import Polygon

################################################################################
# 1) Classes: Area & Region
################################################################################
class Area:
    def __init__(self, area_id, geometry=None):
        self.area_id = area_id
        self.geometry = geometry
        self.nested_ids = set()   # if area encloses smaller polygons

    def __repr__(self):
        return f"Area({self.area_id})"

class Region:
    def __init__(self, target_cardinality):
        self.target_c = target_cardinality
        self.area_ids = set()  # IDs of polygons assigned to this region

    def __repr__(self):
        return f"Region(size={len(self.area_ids)}/{self.target_c})"

    def is_valid(self):
        return len(self.area_ids) == self.target_c

################################################################################
# 2) Build Rook Adjacency with a Spatial Join
#    - We convert polygons to boundary lines
#    - We do an inner sjoin => pairs of boundaries that intersect
#    - For each pair, if the intersection length > eps, we add an edge
################################################################################


def build_rook_adjacency_sjoin(gdf, id_col="GEOID", eps=1e-9):
    import networkx as nx
    import geopandas as gpd
    """
    Builds a Graph for rook adjacency via a GeoDataFrame spatial join.

    1) Convert polygons to boundary lines
    2) sjoin with predicate="intersects"
    3) For each joined row, we measure intersection length of geometry__l and geometry__r.
       If > eps, we add an edge between row[f"{id_col}__l"] and row[f"{id_col}__r"].
    4) This skip corner-only contact by ignoring length <= eps.

    Columns named like {col}__l and {col}__r come from the lsuffix, rsuffix params.
    """

    # 1) Create boundary lines
    boundary_gdf = gdf.copy()
    boundary_gdf["boundary_geom"] = boundary_gdf.geometry.boundary
    boundary_gdf = boundary_gdf.set_geometry("boundary_geom")

    # 2) Spatial join
    joined = gpd.sjoin(
        boundary_gdf,
        boundary_gdf,
        how="inner",
        predicate="intersects",
        lsuffix="_l",
        rsuffix="_r"
    )

    # 3) Build a NetworkX Graph
    G = nx.Graph()
    # Add nodes from original gdf
    for idx, row in gdf.iterrows():
        G.add_node(row[id_col])

    # 4) For each pair in joined, measure intersection
    joined = joined.reset_index(drop=True)
    for i, row in joined.iterrows():
        # skip if same polygon (i.e. left & right are the same record)
        if row[f"{id_col}__l"] == row[f"{id_col}__r"]:
            continue

        # geometry__l, geometry__r => boundary lines from left & right
        # measure intersection length
        length = row["geometry__l"].intersection(row["geometry__r"]).length
        if length > eps:
            left_id = row[f"{id_col}__l"]
            right_id = row[f"{id_col}__r"]
            # add edge
            G.add_edge(left_id, right_id)

    return G



################################################################################
# 3) Helper: create_equal_or_near_equal_cardinalities
################################################################################
def create_equal_or_near_equal_cardinalities(n, p, exact=True):
    """
    Build p cardinalities that sum to n. If exact=True, n%p must be 0.
    Otherwise we distribute leftover among the first 'leftover' partitions.
    """
    if exact:
        if n % p != 0:
            raise ValueError(f"Cannot split n={n} exactly into p={p} equal parts!")
        return [n // p] * p
    else:
        base = n // p
        leftover = n % p
        card_list = []
        for i in range(p):
            if i < leftover:
                card_list.append(base + 1)
            else:
                card_list.append(base)
        return card_list

################################################################################
# 4) PRRP PHASES: grow_region, region_merging_phase, region_splitting_phase
################################################################################
def grow_region(region, A_unassigned, G, seed_area=None, max_region_retries=10):
    """
    Grows the region to region.target_c by randomly picking from boundary neighbors.
    """
    for attempt in range(max_region_retries):
        region.area_ids.clear()
        # pick random seed if none or we are retrying
        chosen_seed = seed_area
        if chosen_seed is None or attempt>0:
            if not A_unassigned:
                return False, region, A_unassigned
            chosen_seed = random.choice(list(A_unassigned))

        region.area_ids.add(chosen_seed)
        A_unassigned.remove(chosen_seed)

        boundary = set()
        for nb in G.neighbors(chosen_seed):
            if nb in A_unassigned:
                boundary.add(nb)

        while len(region.area_ids) < region.target_c and boundary:
            pick = random.choice(list(boundary))
            region.area_ids.add(pick)
            A_unassigned.remove(pick)
            boundary.remove(pick)
            # add new neighbors
            for nb2 in G.neighbors(pick):
                if nb2 in A_unassigned:
                    boundary.add(nb2)

        if len(region.area_ids) == region.target_c:
            return True, region, A_unassigned
        else:
            # revert
            A_unassigned |= region.area_ids
    region.area_ids.clear()
    return False, region, A_unassigned

def region_merging_phase(region, A_unassigned, G):
    """
    Merge leftover unassigned components (except largest) into the new region,
    ensuring leftover remains 1 connected component.
    """
    if not A_unassigned:
        return region, A_unassigned

    subG = G.subgraph(A_unassigned)
    comps = list(nx.connected_components(subG))
    if len(comps) <= 1:
        return region, A_unassigned

    largest_comp = max(comps, key=len)
    for c in comps:
        if c is largest_comp:
            continue
        region.area_ids |= c
        A_unassigned -= c
    return region, A_unassigned

def region_splitting_phase(region, A_unassigned, G, parent_degree_map, max_remove=None):
    """
    If region > target => remove boundary areas
    If region < target => add from boundary of A_unassigned
    """
    target_c = region.target_c
    cur_size = len(region.area_ids)

    if cur_size == target_c:
        return True
    # shrink
    if cur_size > target_c:
        excess = cur_size - target_c
        tries = 0
        while excess>0 and region.area_ids:
            tries += 1
            if max_remove and tries>max_remove:
                return False
            boundary_r = compute_region_boundary(region, A_unassigned, G)
            if not boundary_r:
                return False
            area_id = random.choice(list(boundary_r))
            region.area_ids.remove(area_id)
            A_unassigned.add(area_id)
            excess -= 1
            # fix connectivity
            comps = list(nx.connected_components(G.subgraph(region.area_ids)))
            if len(comps)>1:
                largest_comp = max(comps, key=len)
                for c in comps:
                    if c is largest_comp:
                        continue
                    region.area_ids -= c
                    A_unassigned |= c
                    excess -= len(c)
            if not region.area_ids:
                return False

        if len(region.area_ids)==target_c:
            return True
        if len(region.area_ids)<target_c:
            pass  # proceed to expansion
    # expand
    if len(region.area_ids) < target_c:
        needed = target_c - len(region.area_ids)
        while needed>0 and A_unassigned:
            boundary_u = compute_unassigned_boundary(A_unassigned, G)
            articulation_ids = set(nx.articulation_points(G.subgraph(A_unassigned)))
            boundary_no_artic = [b for b in boundary_u if b not in articulation_ids]
            if not boundary_no_artic:
                return False
            pick = random.choice(boundary_no_artic)
            region.area_ids.add(pick)
            A_unassigned.remove(pick)
            needed -= 1
        return len(region.area_ids)==target_c

    return (len(region.area_ids)==target_c)

def compute_region_boundary(region, A_unassigned, G):
    boundary = set()
    for a_id in region.area_ids:
        for nb in G[a_id]:
            if nb in A_unassigned:
                boundary.add(a_id)
                break
    return boundary

def compute_unassigned_boundary(A_unassigned, G):
    boundary = set()
    for a_id in A_unassigned:
        for nb in G[a_id]:
            if nb not in A_unassigned:
                boundary.add(a_id)
                break
    return boundary

################################################################################
# 5) build_single_solution & prrp_solver
################################################################################
def build_single_solution(area_ids, p_cardinalities, G, parent_degree_map,
                          max_solution_attempts=10):
    """
    Build exactly 1 feasible sample solution with PRRP (grow/merge/split).
    If fails after max_solution_attempts, return (False, []).
    """
    p_card_sorted = list(p_cardinalities)  # or sorted(..., reverse=True)

    for attempt in range(max_solution_attempts):
        A_unassigned = set(area_ids)
        all_regions = []
        success = True

        for c_val in p_card_sorted:
            new_region = Region(c_val)
            # grow
            ok_grow, new_region, A_unassigned = grow_region(new_region, A_unassigned, G)
            if not ok_grow:
                success = False
                break
            # merge
            new_region, A_unassigned = region_merging_phase(new_region, A_unassigned, G)
            # split
            ok_split = region_splitting_phase(new_region, A_unassigned, G, parent_degree_map)
            if not ok_split or not new_region.is_valid():
                success = False
                break
            all_regions.append(new_region)

        if success and len(all_regions) == len(p_card_sorted):
            return True, all_regions

    return False, []

def prrp_solver(area_list, p_cardinalities, M, G, parent_degree_map,
                max_solution_attempts=10, q_cores=1):
    """
    Build M solutions in a simple loop (no parallel).
    """
    solutions = []
    area_ids = [a.area_id for a in area_list]
    for i in range(M):
        ok, regs = build_single_solution(
            area_ids, p_cardinalities, G, parent_degree_map,
            max_solution_attempts=max_solution_attempts
        )
        solutions.append(regs if ok else None)
    return solutions

################################################################################
# 6) MAIN DEMO
################################################################################
def main_test():
    # (A) Load your shapefile into a GeoDataFrame
    # Adjust path to your actual shapefile
    shp_path = "/content/drive/MyDrive/Spatial_project_Dataset/tlgdb_a_42_pa_Block_Group_2020"
    gdf = gpd.read_file(shp_path)
    print("Loaded shapefile. #Records =", len(gdf))
    print(gdf.head())

    # (B) Build adjacency using sjoin-based approach
    id_col = "GEOID"  # or "TRACTCE", etc. Must be unique
    G = build_rook_adjacency_sjoin(gdf, id_col=id_col)
    print("Built adjacency =>", G.number_of_nodes(), "nodes;", G.number_of_edges(), "edges")

    # (C) Create area_list
    area_list = []
    for i, row in gdf.iterrows():
        a_id = row[id_col]
        geom = row["geometry"]
        area_list.append(Area(area_id=a_id, geometry=geom))

    # (D) parent_degree_map => assume no nested polygons
    parent_degree_map = {a.area_id: 0 for a in area_list}

    # (E) Choose p => number of regions
    n = len(area_list)
    p = 4
    # For EXACT equal cardinalities => n%4 must be 0.
    # If not, set exact=False to do near-equal
    p_card = create_equal_or_near_equal_cardinalities(n, p, exact=False)
    print("Cardinalities:", p_card)

    # (F) Build M=2 solutions
    M = 1
    solutions = []
    for i in range(M):
        ok, regs = build_single_solution(
            [a.area_id for a in area_list],
            p_card,
            G,
            parent_degree_map,
            max_solution_attempts=10
        )
        solutions.append(regs if ok else None)

    # (G) Print results
    for i, sol in enumerate(solutions):
        if sol is None:
            print(f"\nSolution #{i} => FAILED to build.")
        else:
            print(f"\nSolution => # of regions = {len(sol)}")
            for j, region in enumerate(sol):
                print(f"  Region {j} => target={region.target_c}, assigned {len(region.area_ids)} areas")


if __name__=="__main__":
    main_test()


Loaded shapefile. #Records = 10173
          GEOID       NAMELSAD       ALAND    AWATER     INTPTLAT  \
0  420010301011  Block Group 1  25936811.0   61286.0  +40.0417045   
1  420010301012  Block Group 2  28839807.0  856724.0  +40.0051064   
2  420010301031  Block Group 1  18036255.0  601611.0  +39.9747338   
3  420010301041  Block Group 1  18706039.0  240326.0  +39.9407610   
4  420010301042  Block Group 2  31389419.0  260773.0  +39.9355824   

      INTPTLON                                           geometry  
0  -77.1190174  POLYGON ((-77.15558 40.05671, -77.15467 40.057...  
1  -77.0706862  POLYGON ((-77.10447 40.00253, -77.10407 40.003...  
2  -77.0284326  POLYGON ((-77.07256 39.9748, -77.07223 39.9749...  
3  -77.0071292  POLYGON ((-77.03642 39.9433, -77.03367 39.9455...  
4  -77.0699731  POLYGON ((-77.09913 39.94784, -77.0975 39.9489...  
Built adjacency => 10173 nodes; 28336 edges
Cardinalities: [2544, 2543, 2543, 2543]

Solution => # of regions = 4
  Region 0 => target=2544, a

In [ ]:
import random
import networkx as nx
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon, Point

###############################################################################
# 1) CLASSES: Area & Region
###############################################################################
class Area:
    def __init__(self, area_id, geometry=None):
        self.area_id = area_id
        self.geometry = geometry
        self.nested_ids = set()  # if area encloses smaller polygons

    def __repr__(self):
        return f"Area({self.area_id})"

class Region:
    def __init__(self, target_cardinality):
        self.target_c = target_cardinality
        self.area_ids = set()  # IDs of polygons assigned to this region

    def __repr__(self):
        return f"Region(size={len(self.area_ids)}/{self.target_c})"

    def is_valid(self):
        return len(self.area_ids) == self.target_c

###############################################################################
# 2) BUILD ADJACENCY (Rook, Queen, KNN)
###############################################################################
from scipy.spatial import KDTree

def build_adjacency(gdf, id_col="GEOID", mode="rook", k=5, eps=1e-9):
    """
    Builds a Graph for adjacency using different modes:
      'rook'  => shared boundary (non-zero line)
      'queen' => shared boundary OR corner (touches)
      'knn'   => each polygon has k nearest neighbors by centroid
    """
    G = nx.Graph()

    # Add nodes
    for i, row in gdf.iterrows():
        G.add_node(row[id_col])

    if mode in ("rook", "queen"):
        # For rook => 'predicate="intersects"' + measure boundary length > eps
        # For queen => 'predicate="touches"' (or some logic). We'll unify them by "intersects" but handle length condition for rook, or skip for queen
        # Easiest approach: build boundary lines, do sjoin, measure intersection length if 'rook'
        boundary_gdf = gdf.copy()
        boundary_gdf["boundary_geom"] = boundary_gdf.geometry.boundary
        boundary_gdf.set_geometry("boundary_geom", inplace=True)

        if mode == "rook":
            # sjoin with 'intersects'
            joined = gpd.sjoin(boundary_gdf, boundary_gdf, how="inner",
                               predicate="intersects", lsuffix="_l", rsuffix="_r")
            joined = joined.reset_index(drop=True)
            for idx, row in joined.iterrows():
                if row[f"{id_col}__l"] == row[f"{id_col}__r"]:
                    continue
                inter_len = row["geometry__l"].intersection(row["geometry__r"]).length
                if inter_len > eps:
                    G.add_edge(row[f"{id_col}__l"], row[f"{id_col}__r"])

        elif mode == "queen":
            # queen => polygons that "touch" in any way (including corners)
            joined = gpd.sjoin(gdf[[id_col, "geometry"]], gdf[[id_col, "geometry"]],
                               how="inner", predicate="touches", lsuffix="_l", rsuffix="_r")
            joined = joined.reset_index(drop=True)
            for idx, row in joined.iterrows():
                if row[f"{id_col}__l"] != row[f"{id_col}__r"]:
                    G.add_edge(row[f"{id_col}__l"], row[f"{id_col}__r"])

    elif mode == "knn":
        # KNN adjacency by centroid
        coords = np.array([list(poly.centroid.coords)[0] for poly in gdf.geometry])
        kd = KDTree(coords)
        for i, c in enumerate(coords):
            # find k neighbors
            distances, neighbors = kd.query(c, k+1)  # includes self
            # skip neighbors[0] which is self
            node_i = gdf.iloc[i][id_col]
            for nb_i in neighbors[1:]:
                node_nb = gdf.iloc[nb_i][id_col]
                G.add_edge(node_i, node_nb)

    return G

###############################################################################
# 3) CARDINALITY METHODS: equal, near-equal, random
###############################################################################
def create_cardinality_list(n, p, mode="equal"):
    """
    mode='equal': near-equal partition of n into p
    mode='random': random partition summing to n
    mode='imbalanced': one large region, rest small
    """
    if mode == "equal":
        base = n // p
        leftover = n % p
        c_list = []
        for i in range(p):
            if i < leftover:
                c_list.append(base+1)
            else:
                c_list.append(base)
        return c_list

    elif mode == "random":
        # randomly partition n into p parts
        c_list = []
        total = n
        for i in range(p-1):
            val = random.randint(1, total-(p-i-1))
            c_list.append(val)
            total -= val
        c_list.append(total)
        return c_list

    elif mode == "imbalanced":
        # e.g. 50% in region 1, then split remainder equally
        big = n//2
        leftover = n - big
        base = leftover // (p-1)
        c_list = [big] + [base]*(p-2) + [leftover - base*(p-2)]
        return c_list

    return None

###############################################################################
# 4) PRRP PHASES: Grow, Merge, Split (same as your code)
###############################################################################
def compute_region_boundary(region, A_unassigned, G):
    boundary = set()
    for a_id in region.area_ids:
        for nb in G[a_id]:
            if nb in A_unassigned:
                boundary.add(a_id)
                break
    return boundary

def compute_unassigned_boundary(A_unassigned, G):
    boundary = set()
    for a_id in A_unassigned:
        for nb in G[a_id]:
            if nb not in A_unassigned:
                boundary.add(a_id)
                break
    return boundary

def grow_region(region, A_unassigned, G, seed_area=None, max_region_retries=10):
    for attempt in range(max_region_retries):
        region.area_ids.clear()
        chosen_seed = seed_area
        if chosen_seed is None or attempt>0:
            if not A_unassigned:
                return False, region, A_unassigned
            chosen_seed = random.choice(list(A_unassigned))

        region.area_ids.add(chosen_seed)
        A_unassigned.remove(chosen_seed)

        boundary = set()
        for nb in G.neighbors(chosen_seed):
            if nb in A_unassigned:
                boundary.add(nb)

        while len(region.area_ids)< region.target_c and boundary:
            pick = random.choice(list(boundary))
            region.area_ids.add(pick)
            A_unassigned.remove(pick)
            boundary.remove(pick)
            for nb2 in G.neighbors(pick):
                if nb2 in A_unassigned:
                    boundary.add(nb2)

        if len(region.area_ids)==region.target_c:
            return True, region, A_unassigned
        else:
            A_unassigned |= region.area_ids
    region.area_ids.clear()
    return False, region, A_unassigned

def region_merging_phase(region, A_unassigned, G):
    if not A_unassigned:
        return region, A_unassigned
    subG = G.subgraph(A_unassigned)
    comps = list(nx.connected_components(subG))
    if len(comps)<=1:
        return region, A_unassigned
    largest_comp = max(comps, key=len)
    for c in comps:
        if c is largest_comp:
            continue
        region.area_ids |= c
        A_unassigned -= c
    return region, A_unassigned

def region_splitting_phase(region, A_unassigned, G, parent_degree_map, max_remove=None):
    target_c = region.target_c
    cur_size = len(region.area_ids)
    if cur_size==target_c:
        return True
    if cur_size>target_c:
        excess = cur_size - target_c
        tries=0
        while excess>0 and region.area_ids:
            tries+=1
            if max_remove and tries>max_remove:
                return False
            boundary_r = compute_region_boundary(region, A_unassigned, G)
            if not boundary_r:
                return False
            area_id = random.choice(list(boundary_r))
            region.area_ids.remove(area_id)
            A_unassigned.add(area_id)
            excess-=1
            comps = list(nx.connected_components(G.subgraph(region.area_ids)))
            if len(comps)>1:
                largest_comp = max(comps, key=len)
                for c in comps:
                    if c is largest_comp:
                        continue
                    region.area_ids -= c
                    A_unassigned |= c
                    excess-=len(c)
            if not region.area_ids:
                return False
        if len(region.area_ids)==target_c:
            return True
        if len(region.area_ids)<target_c:
            pass
    if len(region.area_ids)<target_c:
        needed = target_c - len(region.area_ids)
        while needed>0 and A_unassigned:
            boundary_u = compute_unassigned_boundary(A_unassigned, G)
            articulation_ids = set(nx.articulation_points(G.subgraph(A_unassigned)))
            boundary_no_artic = [b for b in boundary_u if b not in articulation_ids]
            if not boundary_no_artic:
                return False
            pick = random.choice(boundary_no_artic)
            region.area_ids.add(pick)
            A_unassigned.remove(pick)
            needed-=1
        return len(region.area_ids)==target_c
    return (len(region.area_ids)==target_c)

###############################################################################
# 5) build_single_solution & prrp_solver
###############################################################################
class RegionObj:
    def __init__(self, p_card):
        self.p_card = p_card
        self.regions = []

def build_single_solution(area_ids, p_cardinalities, G, parent_degree_map, max_solution_attempts=10):
    p_card_sorted = p_cardinalities[:]
    for attempt in range(max_solution_attempts):
        A_unassigned = set(area_ids)
        all_regions = []
        success = True
        for c_val in p_card_sorted:
            new_region = Region(c_val)
            ok_grow, new_region, A_unassigned = grow_region(new_region, A_unassigned, G)
            if not ok_grow:
                success = False
                break
            new_region, A_unassigned = region_merging_phase(new_region, A_unassigned, G)
            ok_split = region_splitting_phase(new_region, A_unassigned, G, parent_degree_map)
            if not ok_split or not new_region.is_valid():
                success=False
                break
            all_regions.append(new_region)
        if success and len(all_regions)==len(p_card_sorted):
            return True, all_regions
    return False, []

def prrp_solver(area_ids, p_card, G, parent_degree_map, M=2, max_solution_attempts=10):
    solutions = []
    for i in range(M):
        ok, regs = build_single_solution(area_ids, p_card, G, parent_degree_map, max_solution_attempts)
        solutions.append(regs if ok else None)
    return solutions

###############################################################################
# 6) MASTER TEST FUNCTION
###############################################################################
def test_configurations(shapefile_path, id_col="GEOID", adjacency_modes=["rook"],
                       p_values=[4], cardinality_modes=["equal"], M=2):
    """
    Allows testing multiple adjacency modes, p values, and cardinality modes.
    For each combination, we build adjacency, create area_list, run the solver, and print results.
    """

    # Load the data once
    gdf = gpd.read_file(shapefile_path)
    n = len(gdf)
    print(f"Loaded {n} polygons from {shapefile_path}.\n")

    # Build area_list
    area_list = []
    for i, row in gdf.iterrows():
        area_list.append(Area(area_id=row[id_col], geometry=row["geometry"]))

    # By default, no nested polygons => zero
    parent_degree_map = {a.area_id:0 for a in area_list}
    area_ids = [a.area_id for a in area_list]

    # Try each adjacency mode
    for adj_mode in adjacency_modes:
        print(f"=== Building adjacency with mode='{adj_mode}' ===")
        G = build_adjacency(gdf, id_col=id_col, mode=adj_mode)
        print(f"Adjacency => {G.number_of_nodes()} nodes, {G.number_of_edges()} edges.")

        # Now test each p
        for p in p_values:
            # Test each cardinality mode
            for c_mode in cardinality_modes:
                p_card = create_cardinality_list(n, p, mode=c_mode)
                print(f"\n>>> Testing p={p}, cardinality_mode='{c_mode}' => {p_card}")
                solutions = prrp_solver(area_ids, p_card, G, parent_degree_map, M=M)
                for s_idx, sol in enumerate(solutions):
                    if sol is None:
                        print(f"  - Solution #{s_idx} => FAILED")
                    else:
                        print(f"  - Solution #{s_idx} => # regions={len(sol)}")
                        for r_idx, region in enumerate(sol):
                            print(f"     Region {r_idx} => target={region.target_c}, actual={len(region.area_ids)}")

    print("\nAll tests completed.\n")

###############################################################################
# 7) MAIN DEMO
###############################################################################
def main_test():
    # Use your shapefile path here
    shapefile_path = "/content/drive/MyDrive/Spatial_project_Dataset/tlgdb_a_42_pa_Block_Group_2020"
    id_col = "GEOID"

    # Let's test:
    # - adjacency_modes => rook, queen, knn
    # - p_values => 2, 4
    # - cardinality_modes => equal, random, imbalanced
    # - M=1 => 1 solution each
    test_configurations(
        shapefile_path,
        id_col=id_col,
        adjacency_modes=["rook","queen"],  # or ["rook","queen","knn"]
        p_values=[2,4],
        cardinality_modes=["equal","random","imbalanced"],
        M=1
    )

if __name__=="__main__":
    main_test()


Loaded 10173 polygons from /content/drive/MyDrive/Spatial_project_Dataset/tlgdb_a_42_pa_Block_Group_2020.

=== Building adjacency with mode='rook' ===
Adjacency => 10173 nodes, 28336 edges.

>>> Testing p=2, cardinality_mode='equal' => [5087, 5086]
  - Solution #0 => # regions=2
     Region 0 => target=5087, actual=5087
     Region 1 => target=5086, actual=5086

>>> Testing p=2, cardinality_mode='random' => [148, 10025]
  - Solution #0 => # regions=2
     Region 0 => target=148, actual=148
     Region 1 => target=10025, actual=10025

>>> Testing p=2, cardinality_mode='imbalanced' => [5086, 5087]
  - Solution #0 => # regions=2
     Region 0 => target=5086, actual=5086
     Region 1 => target=5087, actual=5087

>>> Testing p=4, cardinality_mode='equal' => [2544, 2543, 2543, 2543]
  - Solution #0 => # regions=4
     Region 0 => target=2544, actual=2544
     Region 1 => target=2543, actual=2543
     Region 2 => target=2543, actual=2543
     Region 3 => target=2543, actual=2543

>>> Testing

In [ ]:
import geopandas as gpd
gdf = gpd.read_file("/content/drive/MyDrive/Spatial_project_Dataset/cb_2015_42_tract_500k")


In [ ]:
print(gdf.columns)
print(gdf.head())



Index(['STATEFP', 'COUNTYFP', 'TRACTCE', 'AFFGEOID', 'GEOID', 'NAME', 'LSAD',
       'ALAND', 'AWATER', 'geometry'],
      dtype='object')
  STATEFP COUNTYFP TRACTCE              AFFGEOID        GEOID    NAME LSAD  \
0      42      001  031101  1400000US42001031101  42001031101  311.01   CT   
1      42      003  051100  1400000US42003051100  42003051100     511   CT   
2      42      003  120400  1400000US42003120400  42003120400    1204   CT   
3      42      003  141300  1400000US42003141300  42003141300    1413   CT   
4      42      003  210700  1400000US42003210700  42003210700    2107   CT   

     ALAND  AWATER                                           geometry  
0  3038356    4281  POLYGON ((-77.03108 39.80239, -77.02262 39.806...  
1   183645       0  POLYGON ((-79.97927 40.44287, -79.97562 40.444...  
2   437417       0  POLYGON ((-79.91616 40.46699, -79.9146 40.4674...  
3   889533       0  POLYGON ((-79.93304 40.43343, -79.93403 40.437...  
4   722950       0  POLYGON ((-8